In [1]:
import tensorflow as tf

from base import fenics as fc
from base import ml

In [2]:
# Set to False to use existing weights for the neural network.
# Set to True to continue training the weights from where we left off last time.
TRAIN = True

In [3]:
for width in (80, 90, 100):
    for depth in (5, 6, 7):
        # DNN hyperparameters
        hidden_units = [width] * depth
        # i.e. the number of fine grid points:
        logits = (ml.fine_grid_fineness.t + 1) * (ml.fine_grid_fineness.x + 1)
        processor = ml.ScaleCenter()
        activation = tf.nn.crelu

        ### Training hyperparameters #1
        drop_rate = 0.0
        drop_type = 'dropout'
        gradient_clip = 1.0  # May be set to None to turn off gradient clipping

        ### Logging/saving hyperparameters
        log_steps = 1000  # How many steps to print the current loss.
        model_dir_base = './saved_models/grid_general/'
        uuid = None
        model_dir = ml.model_dir_str(model_dir_base, hidden_units, logits,
                                     processor=processor, activation=activation, 
                                     uuid=uuid)

        compile_kwargs = {'gradient_clip': gradient_clip,
                          'processor': processor,
                          'model_dir': model_dir}
        dnn_factory = ml.DNNFactory(hidden_units, logits, drop_rate=drop_rate,
                                    drop_type=drop_type, log_steps=log_steps, 
                                    activation=activation, batch_norm=False,
                                    compile_kwargs=compile_kwargs)

        max_steps = 0
        if TRAIN:
            # Start off without using FEniCS solutions because they're so much
            # slower to generate
            gen_one_data = fc.GenGeneralSolutionOnGrid(num_fenics=0)

            ### Training hyperparameters #2
            batch_size = 50
            max_steps += 5000
            learning_rate = 0.001
            with ml.BatchData.context(gen_one_data=gen_one_data, batch_size=batch_size) as train_input_fn:
                dnn = dnn_factory(learning_rate=learning_rate)
                dnn.train(input_fn=train_input_fn, max_steps=max_steps)

            # Start using FEniCS solutions
            gen_one_data = fc.GenGeneralSolutionOnGrid()

            batch_size = 500
            max_steps += 3000
            learning_rate = 0.001
            with ml.BatchData.context(gen_one_data=gen_one_data, batch_size=batch_size) as train_input_fn:
                dnn = dnn_factory(learning_rate=learning_rate)
                dnn.train(input_fn=train_input_fn, max_steps=max_steps)

            batch_size = 500
            max_steps += 2000
            learning_rate = 0.0001
            with ml.BatchData.context(gen_one_data=gen_one_data, batch_size=batch_size) as train_input_fn:
                dnn = dnn_factory(learning_rate=learning_rate)
                dnn.train(input_fn=train_input_fn, max_steps=max_steps)

            batch_size = 500
            max_steps += 1000
            learning_rate = 0.00001
            with ml.BatchData.context(gen_one_data=gen_one_data, batch_size=batch_size) as train_input_fn:
                dnn = dnn_factory(learning_rate=learning_rate)
                dnn.train(input_fn=train_input_fn, max_steps=max_steps)

INFO:tensorflow:Using config: {'_model_dir': './saved_models/grid_general/100x7_121__ScaleCenter_crelu', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 1000, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7faeac1b5198>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Skipping training since max_steps has already saved.
INFO:tensorflow:Using config: {'_model_dir': './saved_models/grid_general/100x7_121__ScaleCenter_crelu', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '

In [5]:
# DNN hyperparameters
hidden_units = [90] * 6
# i.e. the number of fine grid points:
logits = (ml.fine_grid_fineness.t + 1) * (ml.fine_grid_fineness.x + 1)
processor = ml.ScaleCenter()
activation = tf.nn.crelu

### Training hyperparameters #1
drop_rate = 0.0
drop_type = 'dropout'
gradient_clip = 1.0  # May be set to None to turn off gradient clipping

### Logging/saving hyperparameters
log_steps = 1000  # How many steps to print the current loss.
model_dir_base = './saved_models/grid_general/'
uuid = None
model_dir = ml.model_dir_str(model_dir_base, hidden_units, logits,
                             processor=processor, activation=activation, 
                             uuid=uuid)

compile_kwargs = {'gradient_clip': gradient_clip,
                  'processor': processor,
                  'model_dir': model_dir}
dnn_factory = ml.DNNFactory(hidden_units, logits, drop_rate=drop_rate,
                            drop_type=drop_type, log_steps=log_steps, 
                            activation=activation, batch_norm=False,
                            compile_kwargs=compile_kwargs)

max_steps = 0
if TRAIN:
    # Start off without using FEniCS solutions because they're so much
    # slower to generate
    gen_one_data = fc.GenGeneralSolutionOnGrid(num_fenics=0)
    
    ### Training hyperparameters #2
    batch_size = 50
    max_steps += 5000
    learning_rate = 0.001
    with ml.BatchData.context(gen_one_data=gen_one_data, batch_size=batch_size) as train_input_fn:
        dnn = dnn_factory(learning_rate=learning_rate)
        dnn.train(input_fn=train_input_fn, max_steps=max_steps)
    
    # Start using FEniCS solutions
    gen_one_data = fc.GenGeneralSolutionOnGrid()
    
    batch_size = 500
    max_steps += 3000
    learning_rate = 0.001
    with ml.BatchData.context(gen_one_data=gen_one_data, batch_size=batch_size) as train_input_fn:
        dnn = dnn_factory(learning_rate=learning_rate)
        dnn.train(input_fn=train_input_fn, max_steps=max_steps)
        
    batch_size = 500
    max_steps += 2000
    learning_rate = 0.0001
    with ml.BatchData.context(gen_one_data=gen_one_data, batch_size=batch_size) as train_input_fn:
        dnn = dnn_factory(learning_rate=learning_rate)
        dnn.train(input_fn=train_input_fn, max_steps=max_steps)
        
    batch_size = 500
    max_steps += 1000
    learning_rate = 0.00001
    with ml.BatchData.context(gen_one_data=gen_one_data, batch_size=batch_size) as train_input_fn:
        dnn = dnn_factory(learning_rate=learning_rate)
        dnn.train(input_fn=train_input_fn, max_steps=max_steps)

INFO:tensorflow:Using config: {'_model_dir': './saved_models/grid_general/90x6_121__ScaleCenter_crelu', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 1000, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7faea45b64a8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./saved_models/grid_general/90x6_12

Moving new file over differing existing file:
src: /tmp/tmprk0sr40o/dolfin_expression_098da6dbcd24edfd83910aad742c8dad.cpp.gz
dst: /home/mlpatrick000/miniconda/envs/mltt/.cache/dijitso/src/dolfin_expression_098da6dbcd24edfd83910aad742c8dad.cpp.gz
backup: /home/mlpatrick000/miniconda/envs/mltt/.cache/dijitso/src/dolfin_expression_098da6dbcd24edfd83910aad742c8dad.cpp.gz.oldMoving new file over differing existing file:
src: /tmp/tmp7a7uvcco/dolfin_expression_098da6dbcd24edfd83910aad742c8dad.cpp.gz
dst: /home/mlpatrick000/miniconda/envs/mltt/.cache/dijitso/src/dolfin_expression_098da6dbcd24edfd83910aad742c8dad.cpp.gz
backup: /home/mlpatrick000/miniconda/envs/mltt/.cache/dijitso/src/dolfin_expression_098da6dbcd24edfd83910aad742c8dad.cpp.gz.old



INFO:tensorflow:Saving checkpoints for 5000 into ./saved_models/grid_general/90x6_121__ScaleCenter_crelu/model.ckpt.


Moving new file over differing existing file:
src: /tmp/tmpp_7935lm/dolfin_expression_098da6dbcd24edfd83910aad742c8dad.cpp.gz
dst: /home/mlpatrick000/miniconda/envs/mltt/.cache/dijitso/src/dolfin_expression_098da6dbcd24edfd83910aad742c8dad.cpp.gz
backup: /home/mlpatrick000/miniconda/envs/mltt/.cache/dijitso/src/dolfin_expression_098da6dbcd24edfd83910aad742c8dad.cpp.gz.old
Backup file exists, overwriting.
Moving new file over differing existing file:
src: /tmp/tmp1iohvjvd/dolfin_expression_cb1d29bd0cfe3d68bde7876e7984ac94.cpp.gz
dst: /home/mlpatrick000/miniconda/envs/mltt/.cache/dijitso/src/dolfin_expression_cb1d29bd0cfe3d68bde7876e7984ac94.cpp.gz
backup: /home/mlpatrick000/miniconda/envs/mltt/.cache/dijitso/src/dolfin_expression_cb1d29bd0cfe3d68bde7876e7984ac94.cpp.gz.old
Moving new file over differing existing file:
src: /tmp/tmpl67v5vst/dolfin_expression_098da6dbcd24edfd83910aad742c8dad.cpp.gz
dst: /home/mlpatrick000/miniconda/envs/mltt/.cache/dijitso/src/dolfin_expression_098da6dbcd

INFO:tensorflow:loss = 0.0067702364, step = 5001
INFO:tensorflow:Saving checkpoints for 5093 into ./saved_models/grid_general/90x6_121__ScaleCenter_crelu/model.ckpt.
INFO:tensorflow:Saving checkpoints for 5215 into ./saved_models/grid_general/90x6_121__ScaleCenter_crelu/model.ckpt.
INFO:tensorflow:Saving checkpoints for 5339 into ./saved_models/grid_general/90x6_121__ScaleCenter_crelu/model.ckpt.
INFO:tensorflow:Saving checkpoints for 5462 into ./saved_models/grid_general/90x6_121__ScaleCenter_crelu/model.ckpt.
INFO:tensorflow:Saving checkpoints for 5585 into ./saved_models/grid_general/90x6_121__ScaleCenter_crelu/model.ckpt.
INFO:tensorflow:Saving checkpoints for 5705 into ./saved_models/grid_general/90x6_121__ScaleCenter_crelu/model.ckpt.
INFO:tensorflow:Saving checkpoints for 5801 into ./saved_models/grid_general/90x6_121__ScaleCenter_crelu/model.ckpt.
INFO:tensorflow:Saving checkpoints for 5920 into ./saved_models/grid_general/90x6_121__ScaleCenter_crelu/model.ckpt.
INFO:tensorflow

Moving new file over differing existing file:
src: /tmp/tmpunwd319t/dolfin_expression_7d94a43af0f9677137393cfbe4011d7e.cpp.gz
dst: /home/mlpatrick000/miniconda/envs/mltt/.cache/dijitso/src/dolfin_expression_7d94a43af0f9677137393cfbe4011d7e.cpp.gz
backup: /home/mlpatrick000/miniconda/envs/mltt/.cache/dijitso/src/dolfin_expression_7d94a43af0f9677137393cfbe4011d7e.cpp.gz.old
Moving new file over differing existing file:
src: /tmp/tmp49bx2c_r/dolfin_expression_7d94a43af0f9677137393cfbe4011d7e.cpp.gz
dst: /home/mlpatrick000/miniconda/envs/mltt/.cache/dijitso/src/dolfin_expression_7d94a43af0f9677137393cfbe4011d7e.cpp.gz
backup: /home/mlpatrick000/miniconda/envs/mltt/.cache/dijitso/src/dolfin_expression_7d94a43af0f9677137393cfbe4011d7e.cpp.gz.old


INFO:tensorflow:Saving checkpoints for 8000 into ./saved_models/grid_general/90x6_121__ScaleCenter_crelu/model.ckpt.
INFO:tensorflow:loss = 0.003061695, step = 8001
INFO:tensorflow:Saving checkpoints for 8092 into ./saved_models/grid_general/90x6_121__ScaleCenter_crelu/model.ckpt.
INFO:tensorflow:Saving checkpoints for 8217 into ./saved_models/grid_general/90x6_121__ScaleCenter_crelu/model.ckpt.
INFO:tensorflow:Saving checkpoints for 8340 into ./saved_models/grid_general/90x6_121__ScaleCenter_crelu/model.ckpt.
INFO:tensorflow:Saving checkpoints for 8463 into ./saved_models/grid_general/90x6_121__ScaleCenter_crelu/model.ckpt.
INFO:tensorflow:Saving checkpoints for 8587 into ./saved_models/grid_general/90x6_121__ScaleCenter_crelu/model.ckpt.
INFO:tensorflow:Saving checkpoints for 8711 into ./saved_models/grid_general/90x6_121__ScaleCenter_crelu/model.ckpt.
INFO:tensorflow:Saving checkpoints for 8834 into ./saved_models/grid_general/90x6_121__ScaleCenter_crelu/model.ckpt.
INFO:tensorflow: